##### ARTI 560 - Computer Vision  
## Image Classification using Transfer Learning - Exercise 

### Objective

In this exercise, you will:

1. Select another pretrained model (e.g., VGG16, MobileNetV2, or EfficientNet) and fine-tune it for CIFAR-10 classification.  
You'll find the pretrained models in [Tensorflow Keras Applications Module](https://www.tensorflow.org/api_docs/python/tf/keras/applications).

2. Before training, inspect the architecture using model.summary() and observe:
- Network depth
- Number of parameters
- Trainable vs Frozen layers

3. Then compare its performance with ResNet and the custom CNN.

### Questions:

- Which model achieved the highest accuracy?
- Which model trained faster?
- How might the architecture explain the differences?

Student Name: Fajer Alzamanan
Student ID: 2220006879
Section No. : 10FA01

### Exercise Description

This notebook presents an implementation of transfer learning using **MobileNetV2** for CIFAR-10 image classification. The model architecture is analyzed in terms of network depth, number of parameters, and trainable versus frozen layers. The performance is then compared with **ResNet50V2** and a **Custom CNN** model to evaluate accuracy, computational efficiency, and architectural differences.

---


In [1]:
import time
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()
y_train = y_train.squeeze()
y_test  = y_test.squeeze()

x_train = x_train.astype("float32")
x_test  = x_test.astype("float32")


/opt/anaconda3/envs/cv_lab/lib/python3.13/site-packages/keras/src/datasets/cifar.py:18: VisibleDeprecationWarning: dtype(): align should be passed as Python or NumPy boolean but got `align=0`. Did you mean to pass a tuple to create a subarray type? (Deprecated NumPy 2.4)
  d = cPickle.load(f, encoding="bytes")


In [2]:
data_augmentation = keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.05),
    layers.RandomZoom(0.1),
], name="augmentation")


In [3]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

base_model = MobileNetV2(
    include_top=False,
    weights="imagenet",
    input_shape=(224, 224, 3)
)

base_model.trainable = False  # Freeze

inputs = keras.Input(shape=(32, 32, 3))
x = data_augmentation(inputs)
x = layers.Resizing(224, 224, interpolation="bilinear")(x)
x = layers.Lambda(preprocess_input)(x)

x = base_model(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.2)(x)
outputs = layers.Dense(10)(x)  # logits (مثل lab02)

mobile_model = keras.Model(inputs, outputs, name="MobileNetV2_TL")

mobile_model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"]
)

mobile_model.summary()
print("Trainable weights:", len(mobile_model.trainable_weights))
print("Non-trainable weights:", len(mobile_model.non_trainable_weights))


Model: "MobileNetV2_TL"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ augmentation (Sequential)       │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ resizing (Resizing)             │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lambda (Lambda)                 │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 10)             │        12,810 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,270,794 (8.66 MB)

 Trainable params: 12,810 (50.04 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

Trainable weights: 2
Non-trainable weights: 260


In [4]:
t0 = time.time()

history_m = mobile_model.fit(
    x_train, y_train,
    validation_split=0.1,
    epochs=3,
    batch_size=64,
    verbose=1
)

t1 = time.time()

m_loss, m_acc = mobile_model.evaluate(x_test, y_test, verbose=0)
m_time = t1 - t0

print(f"MobileNetV2 (frozen) test acc: {m_acc:.4f}")
print(f"Training time (s): {m_time:.1f}")


Epoch 1/3
704/704 ━━━━━━━━━━━━━━━━━━━━ 363s 513ms/step - accuracy: 0.6455 - loss: 1.0117 - val_accuracy: 0.8054 - val_loss: 0.5746
Epoch 2/3
704/704 ━━━━━━━━━━━━━━━━━━━━ 563s 801ms/step - accuracy: 0.7220 - loss: 0.7967 - val_accuracy: 0.8238 - val_loss: 0.5042
Epoch 3/3
704/704 ━━━━━━━━━━━━━━━━━━━━ 360s 512ms/step - accuracy: 0.7307 - loss: 0.7701 - val_accuracy: 0.8324 - val_loss: 0.4920
MobileNetV2 (frozen) test acc: 0.8233
Training time (s): 1287.3


In [5]:
base_model.trainable = True
for layer in base_model.layers[:-30]:
    layer.trainable = False

mobile_model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-5),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"]
)

t0 = time.time()

history_m_ft = mobile_model.fit(
    x_train, y_train,
    validation_split=0.1,
    epochs=3,
    batch_size=64,
    verbose=1
)

t1 = time.time()

mft_loss, mft_acc = mobile_model.evaluate(x_test, y_test, verbose=0)
mft_time = t1 - t0

print(f"MobileNetV2 (fine-tuned) test acc: {mft_acc:.4f}")
print(f"Fine-tuning time (s): {mft_time:.1f}")


Epoch 1/3
704/704 ━━━━━━━━━━━━━━━━━━━━ 516s 728ms/step - accuracy: 0.6945 - loss: 0.8966 - val_accuracy: 0.8256 - val_loss: 0.5073
Epoch 2/3
704/704 ━━━━━━━━━━━━━━━━━━━━ 505s 718ms/step - accuracy: 0.7561 - loss: 0.6991 - val_accuracy: 0.8398 - val_loss: 0.4628
Epoch 3/3
704/704 ━━━━━━━━━━━━━━━━━━━━ 507s 720ms/step - accuracy: 0.7814 - loss: 0.6306 - val_accuracy: 0.8498 - val_loss: 0.4182
MobileNetV2 (fine-tuned) test acc: 0.8521
Fine-tuning time (s): 1528.7


In [6]:
def get_param_counts(model):
    trainable = sum(np.prod(w.shape) for w in model.trainable_weights)
    frozen = sum(np.prod(w.shape) for w in model.non_trainable_weights)
    total = trainable + frozen
    return total, trainable, frozen

total_p, train_p, frozen_p = get_param_counts(mobile_model)
print("Total params:", total_p)
print("Trainable params:", train_p)
print("Frozen params:", frozen_p)


Total params: 2270794
Trainable params: 1539210
Frozen params: 731584


In [8]:
import pandas as pd

comparison = pd.DataFrame([
    {
        "Model": "Custom CNN",
        "Frozen Accuracy": "-",
        "Fine-tuned Accuracy": 0.8742,
        "Total Params": "Small (~1M)",
        "Trainable Params": "All",
        "Frozen Params": 0
    },
    {
        "Model": "ResNet50V2",
        "Frozen Accuracy": 0.8742,
        "Fine-tuned Accuracy": 0.9162,
        "Total Params": 23585290,
        "Trainable Params": 20490,
        "Frozen Params": 23564800
    },
    {
        "Model": "MobileNetV2",
        "Frozen Accuracy": 0.8233,
        "Fine-tuned Accuracy": 0.8521,
        "Total Params": 2270794,
        "Trainable Params": 1539210,
        "Frozen Params": 731584
    }
])

comparison


,Model,Frozen Accuracy,Fine-tuned Accuracy,Total Params,Trainable Params,Frozen Params
0,Custom CNN,-,0.8742,Small (~1M),All,0
1,ResNet50V2,0.8742,0.9162,23585290,20490,23564800
2,MobileNetV2,0.8233,0.8521,2270794,1539210,731584


## Model Comparison

| Model       | Frozen Accuracy | Fine-tuned Accuracy | Total Parameters | Trainable Parameters | Frozen Parameters |
| ----------- | --------------- | ------------------- | ---------------- | -------------------- | ----------------- |
| Custom CNN  | —               | 0.8742              | Small (~1M)      | All                  | 0                 |
| ResNet50V2  | 0.8742          | 0.9162              | 23,585,290       | 20,490               | 23,564,800        |
| MobileNetV2 | 0.8233          | 0.8521              | 2,270,794        | 1,539,210            | 731,584           |

---

## Questions Answers

### 1. Which model achieved the highest accuracy?

Among the three models, **ResNet50V2** achieved the highest accuracy with approximately **91.6%** after fine-tuning. This superior performance is mainly due to its deep architecture and residual connections, which enable more effective feature extraction compared to the other models.

---

### 2. Which model trained faster?

The **Custom CNN** trained the fastest because it has a simpler architecture and significantly fewer parameters compared to the pretrained models. **MobileNetV2** required moderate training time due to its lightweight design, while **ResNet50V2** took the longest training time because of its large number of parameters and deep network structure.

---

### 3. How might the architecture explain the differences?

The differences in performance can be explained by the architectural complexity of each model. The **Custom CNN** is relatively shallow with fewer parameters, which limits its ability to learn complex features but allows faster training. **ResNet50V2** is a very deep network with residual connections that improve gradient flow and feature learning, resulting in higher accuracy but increased computational cost. **MobileNetV2** uses depthwise separable convolutions, which significantly reduce the number of parameters and computations while maintaining good feature extraction capability, providing a balance between accuracy and efficiency. These results also demonstrate the advantage of transfer learning compared to training a model from scratch.


## Additional Observations

The parameter statistics were obtained from the `model.summary()` outputs. ResNet50V2 has significantly more parameters than MobileNetV2 and the custom CNN, which increases computational cost but improves feature learning. MobileNetV2 reduces complexity using depthwise separable convolutions, providing efficient performance with fewer parameters. Fine-tuning improved the accuracy of the pretrained models compared to the frozen configuration, highlighting the benefit of transfer learning. Overall, the results demonstrate the trade-off between model complexity, efficiency, and accuracy.
